In [3]:
import tensorflow as tf
import importlib
import os
import loss_functions
import dataprocessor as dataprocessor
import model
import pickle
import change_point_detection
from tqdm import tqdm
from heartrate import trace

def reload_custom_libs():
    importlib.reload(loss_functions)
    importlib.reload(dataprocessor)
    importlib.reload(change_point_detection)
    importlib.reload(model)



reload_custom_libs()
from change_point_detection import *
from loss_functions import *
from model import *
from dataprocessor import *

macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126
folder_path = "data"
files = [f for f in os.listdir(folder_path) if f.endswith(".csv")][:10]

In [4]:
# 获得高斯变点片段
# test = (20080101, 20090101)
gaussion_process_list = []
# for i in tqdm(range(15)):
#     first_date, last_date = (test[0] + 10000 * i, test[1] + 10000 * i)
#     # 处理数据
#     data_list = process_data_list(files, macd_timescales, rtn_timescales, first_date=first_date, last_date=last_date)
#     print(f"{first_date}_{last_date}数据处理完成")
#     # 获得断点分割片段数据
#     try:
#         gaussion_process_list = get_segment_list(data_list=data_list)
#     except:
#         continue
#     with open(f'segments/{first_date}_{last_date}.pkl', 'wb') as f:
#         pickle.dump(gaussion_process_list, f)
#     # # 读取数据
#     # with open(f'segments/{first_date}_{last_date}.pkl', 'rb') as f:
#     #     file = pickle.load(f)
#     #     gaussion_process_list.extend(file)


data_list = process_data_list(files, macd_timescales, rtn_timescales, last_date=20211231)

pkl_files = [f for f in os.listdir("segments") if f.endswith(".pkl")]
for file in pkl_files:
    with open("segments/" + file, 'rb') as file:
        data = pickle.load(file)
        gaussion_process_list.extend(data)

# 训练数据预处理

In [5]:
# 生成数据： target_set 和 context_set
asset_num, context_num = 10, 20
target_set, labels, map = generate_tensors(data_list, time_steps=timesteps, encoder_type = "one-hot", return_map=True)
with open(f'map.pkl', 'wb') as f:
    pickle.dump(map, f)
with open(f'map.pkl', 'rb') as f:
    map = pickle.load(f)
target_set, context_set, labels = gaussian_data_binder(
    data_list,
    target_set,
    labels,
    map=map,
    asset_num=asset_num,
    context_num=context_num,
    gaussion_process_list=gaussion_process_list,
)

# 设置参数
target_std = tf.cast(5e-2, tf.float64)
hidden_dim = 64  # 128
warm_up = 63
features_len = len(macd_timescales) + len(rtn_timescales)

生成张量, 并对类别信息进行one-hot 编码: 100%|██████████| 10/10 [00:06<00:00,  1.54it/s]


one-hot 编码中...


# 训练模型初始化

In [6]:
reload_custom_libs()
from change_point_detection import *
from loss_functions import *
from model import *
from dataprocessor import *

In [7]:
# declare 数据, 初始化数据集
x, s = target_set[0], target_set[-1]
x_c_rtn, x_c, s_c = context_set[0], context_set[0][:, :, :, 1:], context_set[-1]
dataset = tf.data.Dataset.from_tensor_slices((x_c, x_c_rtn, s_c, x, s, labels))

timesteps = x.shape[-2]
features_len = x.shape[-1]
encoding_size = s.shape[-1]
x_shape = (None, asset_num, timesteps, features_len)
s_shape = (None, asset_num, timesteps, encoding_size)

# 初始化模型
xtrend_model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads=4, dropout_rate=0.4)
xtrend_model.build((x_shape, s_shape))

# 训练模型

In [8]:
# # 训练模型脚本 depreciated
# def train(model, dataset: tf.data.Dataset, batch_num: int, num_epochs: int, alpha: float):

#     optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
#     dataset = dataset.shuffle(buffer_size=10000).batch(batch_num)

#     for epoch in range(num_epochs):
#         iter_count = 0
#         for batch_data in tqdm(dataset, desc=f"训练中...epoch{epoch}"):
#             # 对每一个批次进行处理
#             x_c, x_c_rtn, s_c, x, s, labels = batch_data
#             with tf.GradientTape() as tape:
#                 result = model(x_c, x_c_rtn, s_c, x, s)
#                 joint_loss, mle, sharpe = joint_loss_function(
#                     result, labels, target_std, warm_up, alpha=alpha
#                 )
#             grads = tape.gradient(joint_loss, model.trainable_variables)
#             optimizer.apply_gradients(zip(grads, model.trainable_variables))

#             iter_count += 1
#             if iter_count % 1 == 0 and True:
#                 print(
#                     f"Epoch {epoch+1}/{num_epochs}, Iteration {iter_count}, Loss: {joint_loss.numpy()}, MLE: {mle.numpy()}, Sharpe: {sharpe.numpy()}"
#                 )

#         print(f"Epoch {epoch+1}/{num_epochs}, Loss: {joint_loss.numpy()}")

# # 初始化模型
# model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads=4)
# model.build((x_shape, s_shape))
# result = train(model, dataset, batch_num=64, num_epochs=100, alpha = 1e-2)

In [9]:
# 训练模型
def train(model, dataset: tf.data.Dataset, batch_num: int, num_epochs: int, alpha: float, validation_split: float = 0.2, early_stopping_patience: int = 10):
    # 将数据集拆分为训练集和验证集
    dataset = dataset.shuffle(buffer_size=10000)
    total_size = len(dataset)
    val_size = int(total_size * validation_split)
    train_size = total_size - val_size

    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size).batch(batch_num)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm=100.0)
    best_val_loss = np.inf
    
    for epoch in range(num_epochs):
        # 每个 epoch 开始的时候重新shuffle数据集
        train_dataset = dataset.take(train_size).shuffle(buffer_size=10000).batch(batch_num)
        for batch_data in tqdm(train_dataset, desc=f"训练中...epoch{epoch}"):
            # 对每一个批次进行处理
            x_c, x_c_rtn, s_c, x, s, labels = batch_data
            with tf.GradientTape() as tape:
                result = model(x_c, x_c_rtn, s_c, x, s)
                joint_loss, mle, sharpe = joint_loss_function(
                    result, labels, target_std, warm_up, alpha=alpha
                )
            grads = tape.gradient(joint_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # 在每个epoch结束后，计算验证集的joint loss
        val_joint_loss = 0.0
        val_mle = 0.0
        val_sharpe = 0.0
        val_steps = 0
        for val_batch_data in val_dataset:
            x_c, x_c_rtn, s_c, x, s, labels = val_batch_data
            result = model(x_c, x_c_rtn, s_c, x, s)
            joint_loss, mle, sharpe = joint_loss_function(
                result, labels, target_std, warm_up, alpha=alpha
            )
            val_joint_loss += joint_loss.numpy()
            val_mle += mle.numpy()
            val_sharpe += sharpe.numpy()
            val_steps += 1

        val_joint_loss /= val_steps
        val_mle /= val_steps
        val_sharpe /= val_steps
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Joint Loss: {val_joint_loss:.2f}, Validation MLE: {val_mle:.2f}, Validation Sharpe: {val_sharpe:.2f}")
        
        # 早停检查
        if val_joint_loss < best_val_loss:
            best_val_loss = val_joint_loss
            patience_counter = 0
            model.save(f"model/{epoch}_loss_{val_joint_loss:.2f}.keras")
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch+1}")
                break



In [10]:
# 训练模型
train(xtrend_model, dataset, batch_num=64, num_epochs=500, alpha=1e-3)

训练中...epoch0:   0%|          | 0/31 [00:00<?, ?it/s]

训练中...epoch0: 100%|██████████| 31/31 [03:42<00:00,  7.19s/it]


Epoch 1/500, Validation Joint Loss: 0.15, Validation MLE: 0.06, Validation Sharpe: 0.09


训练中...epoch1: 100%|██████████| 31/31 [03:41<00:00,  7.14s/it]


Epoch 2/500, Validation Joint Loss: 0.04, Validation MLE: 0.06, Validation Sharpe: -0.02


...

Epoch 10/500, Validation Joint Loss: 0.20, Validation MLE: 0.06, Validation Sharpe: 0.14


训练中...epoch10: 100%|██████████| 31/31 [06:56<00:00, 13.43s/it]


Epoch 11/500, Validation Joint Loss: 0.05, Validation MLE: 0.06, Validation Sharpe: -0.01


训练中...epoch11: 100%|██████████| 31/31 [07:00<00:00, 13.55s/it]


Epoch 12/500, Validation Joint Loss: 0.59, Validation MLE: 0.06, Validation Sharpe: 0.53
Early stopping at epoch 12


In [11]:
model.save(f"final_model.keras")

AttributeError: module 'model' has no attribute 'save'